#Traductor para Gregos

importamos las librerias, haremos la red neuronal con tensorflow y keras

In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization


Descargamos un dataset que proporciona tensorflow de español y ingles

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

Leemos el dataset y Creamos pares de cada frase con su significado en inlges y español.

se usa frases para que el modelo entienda el contexto en el que se usa, no es lo mismo decir



```
No, te pertenece
```
a

```
No te pertenece
```

ambas palabaras tienen significados opuestos que depende mas que de las palabras que se usen , que contexto tiene toda la frase


In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))
for _ in range(5):
    print(random.choice(text_pairs))


('As soon as I sat down, I fell asleep.', '[start] En cuanto tomé asiento, caí dormido. [end]')
('We regret his death.', '[start] Lamentamos su fallecimiento. [end]')
('Two plus two makes four.', '[start] Dos más dos es cuatro. [end]')
('Think of your family.', '[start] ¡Piense en su familia! [end]')
('What did you buy for your boyfriend?', '[start] ¿Qué le compraste a tu amigo? [end]')


Como dije entregas anteriores una buena practica es separar los datos en 2 conjuntos , uno de entrenamiento ,validacion y otro de prueba para mejorar el model, bueno aqui estoy haciendo eso


In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


definimos nuestros hiperparametros

In [ ]:
vocab_size = 15000
sequence_length = 20
batch_size = 64

Hacemos una funcion que nos limpiara los datos cuando los usemos para vectorizarlos con la funcion 

```
custom_standardization()
```
esta funcion convierte todo a minusculas y elimina algunos signos de puntuacion



In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

Vectorizamos las palabras, este proceso consiste en dividir el string en tokens, construye un vocabulario y como en las redes neuronales no se pueden usar strings, construye un vocabulario como una lista y usa el indice de la palabra para entrenar la red, luego hace una matriz de dispercion para ver que tan parecidas son las palabras

![link text](https://www.tensorflow.org/images/tutorials/transformer/attention_map_portuguese.png?hl=es-419)

matriz de dispercion de ejemplo de ingles a portuges, se observa que mientras mas parecidas las palabras su color en la matriz es mas brillante 

In [ ]:
eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Hacemos nuestra implementacion de los transoformes, los transoformes son un arquitectura de redes neuronales,muy potentes por la "atencion", es la pieza principal de chatgpt.

imagen de la nn que represnta los transformers
![trasnformer](https://www.tensorflow.org/images/tutorials/transformer/transformer.png)


dividimos el transformer en 2, uno para codificar y otro para decodificar, en otros codigos usan solo un transoforme que codifique y decodifique, yo lo prefiero en 2 por orden en el codigo

¿que son los multihead attention?

son capas de las redes de la red que examinan la relacion entre las palabras

¿que son los softmax?

otra capa de la red que crea probabilidades, esto es importante usarlo en claisificacion para esocjer las opciones con mayor probabilidad

¿que son los linear?

son capas que hacen un proceso wx+b a todos los elementos de un vector


## Transoformer Codificador 

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


## decodificador del Transformer  

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

Terminamos de ensablar el model, antes habiamos hecho los transoformers , ya juntamos el transoformer al resto del la NN.

al final llamamos la funcion donde hacemos el ensamble y le pasamos los parametros de capas de entrada y salida

In [ ]:
def create_model(embed_dim ,latent_dim,num_heads,vocab_size,sequence_length):
  encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
  x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
  encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
  encoder = keras.Model(encoder_inputs, encoder_outputs)
  decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
  encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
  x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
  x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
  x = layers.Dropout(0.5)(x)
  decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
  decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
  decoder_outputs = decoder([decoder_inputs, encoder_outputs])
  transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
  )
  return transformer

embed_dim = 256
latent_dim = 2048
num_heads = 8
transormer=create_model(embed_dim ,latent_dim,num_heads,vocab_size,sequence_length)

Ya es hora de entrenar el modelo

In [ ]:
epochs = 10#30  epochs es para mejor calidad

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range():
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
print(translated)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 256)   3155456     ['positional_embedding_

Finalmente ya tenemos el traductor

ingresa una palabra en ingles , para hacer lo de español a ingles se hace el proceso al revez.
 

In [ ]:
txt=input()
print(txt)
translated = decode_sequence(txt)
print(translated)

how are you?
how are you?
[start] cómo son usted [end]
